In [6]:
import pandas as pd
import glob
import os

# 1. Import Company dataset
print("\n[Step 1] Importing S&P 500 Companies Dataset...")
companies_file = "stock_dataset/sp500_companies.csv"
companies_df = pd.read_csv(companies_file)
print(f"Initial Size (Companies): {companies_df.shape}")

# 2. Filter down to Technology sector only
print("\n[Step 2] Filtering Technology Sector Companies...")
technology_companies_df = companies_df[companies_df["Sector"] == "Technology"]
print(f"Final Size (Technology Companies): {technology_companies_df.shape}")

# 3. Get List of symbols for Technology stocks
tech_symbols = technology_companies_df["Symbol"].unique()

# 4. Import Stock dataset
print("\n[Step 3] Importing S&P 500 Stock Trading Dataset...")
stocks_file = "stock_dataset/sp500_stocks.csv"
stocks_df = pd.read_csv(stocks_file, low_memory=False, parse_dates=["Date"])
print(f"Initial Size (Stock Records): {stocks_df.shape}")

# 5. Filter stock records to only Technology companies
print("\n[Step 4] Filtering Stock Records for Technology Companies...")
tech_stocks_df = stocks_df[stocks_df["Symbol"].isin(tech_symbols)]
print(f"Final Size (Filtered Stock Records): {tech_stocks_df.shape}")

# 6. Import Macroeconomic dataset
print("\n[Step 5] Importing Macroeconomic Dataset...")
macro_economic_file = "stock_dataset/macroeconomic.csv"
macro_economic_df = pd.read_csv(macro_economic_file, parse_dates=["DATE"], dayfirst=True)
print(f"Initial Size (Macroeconomic Records): {macro_economic_df.shape}")

# 7. Expand Macroeconomic data to daily frequency
print("\n[Step 6] Expanding Macroeconomic Data to Daily Frequency...")
start_date = macro_economic_df["DATE"].min()
end_date = macro_economic_df["DATE"].max()
daily_dates = pd.date_range(start=start_date, end=end_date, freq="D")
macro_economic_df = macro_economic_df.set_index("DATE").reindex(daily_dates).interpolate(method="linear").reset_index()
macro_economic_df.rename(columns={"index": "Date"}, inplace=True)
print(f"Final Size (Interpolated Daily Macroeconomic Records): {macro_economic_df.shape}")

# 8. Merge Stock and Macroeconomic datasets
print("\n[Step 7] Merging Stock and Macroeconomic Datasets...")
merged_df = tech_stocks_df.merge(macro_economic_df, on="Date", how="left")
print(f"Size After Merge: {merged_df.shape}")

# 9. Drop rows with missing values
print("\n[Step 8] Dropping Missing Records")
merged_df_clean = merged_df.dropna()
print(f"Final Cleaned Size: {merged_df_clean.shape}")

merged_df_clean.describe()
#merged_df_clean.to_csv("stock_dataset/sp500_stocks_with_macro_clean.csv", index=False)



[Step 1] Importing S&P 500 Companies Dataset...
Initial Size (Companies): (502, 16)

[Step 2] Filtering Technology Sector Companies...
Final Size (Technology Companies): (82, 16)

[Step 3] Importing S&P 500 Stock Trading Dataset...
Initial Size (Stock Records): (1891536, 8)

[Step 4] Filtering Stock Records for Technology Companies...
Final Size (Filtered Stock Records): (308976, 8)

[Step 5] Importing Macroeconomic Dataset...
Initial Size (Macroeconomic Records): (241, 15)

[Step 6] Expanding Macroeconomic Data to Daily Frequency...
Final Size (Interpolated Daily Macroeconomic Records): (7306, 15)

[Step 7] Merging Stock and Macroeconomic Datasets...
Size After Merge: (308976, 22)

[Step 8] Dropping Missing Records
Final Cleaned Size: (62895, 22)


,Date,Adj Close,Close,High,Low,Open,Volume,UNRATE(%),CONSUMER CONF INDEX,PPI-CONST MAT.,...,INFLATION(%),MORTGAGE INT. MONTHLY AVG(%),MED HOUSEHOLD INCOME,CORP. BOND YIELD(%),MONTHLY HOME SUPPLY,% SHARE OF WORKING POPULATION,GDP PER CAPITA,QUARTERLY REAL GDP,QUARTERLY GDP GROWTH RATE (%),CSUSHPISA
count,62895,62895.000000,62895.000000,62895.000000,62895.000000,62895.000000,6.289500e+04,62895.000000,62895.000000,62895.000000,...,62895.000000,62895.000000,62895.000000,62895.000000,62895.000000,62895.000000,62895.000000,62895.00000,62895.000000,62895.000000
mean,2016-07-24 20:27:25.027426560,49.018361,52.269326,52.923529,51.592162,52.266286,3.460345e+07,6.053459,97.187728,230.557046,...,2.198947,3.888244,59502.553954,3.706669,5.625044,65.872988,58828.579295,17702.06850,0.518660,154.838348
min,2010-01-04 00:00:00,0.203593,0.222000,0.226250,0.216250,0.218000,0.000000e+00,3.500000,39.986667,190.483871,...,-0.199517,2.684000,49276.000000,2.140000,3.306452,64.924129,47797.000000,15456.05900,-8.937251,120.796600
25%,2013-08-07 00:00:00,13.174779,15.450000,15.658125,15.250000,15.450000,2.041450e+06,4.193333,78.183871,209.541935,...,1.267895,3.524855,53585.000000,3.313548,5.035484,65.086278,53281.000000,16594.74300,0.323373,141.073097
50%,2016-10-13 00:00:00,30.741734,35.130001,35.630001,34.700001,35.139999,5.477300e+06,5.400000,97.790000,215.809677,...,1.794036,3.913823,59039.000000,3.814839,5.464516,65.880715,58263.000000,17724.48900,0.589180,150.782065
75%,2019-08-14 00:00:00,61.689999,66.055000,66.847000,65.200497,66.000000,1.491900e+07,7.777419,121.667742,237.267742,...,2.418063,4.305357,67521.000000,4.058387,6.100000,66.550618,63701.000000,18767.77800,0.817923,169.481667
max,2022-04-29 00:00:00,411.220001,411.220001,413.890015,407.760010,413.220001,3.692928e+09,14.700000,138.350000,352.248533,...,8.559986,5.213500,68703.000000,5.350000,9.235484,67.128435,74737.000000,19806.29000,7.547535,184.599000
std,NaN,56.459594,56.683658,57.385631,55.961200,56.693837,1.305347e+08,2.189739,25.779402,36.129983,...,1.696022,0.569605,6842.666337,0.719699,1.076781,0.755859,6664.853482,1241.85717,1.732607,17.740659
